# Chain of Responsibility: Broken Chain

> Invoking the chain implicitly

In this scenario, we'll improve upon our previous creature modifiers by having them applied without explicitly calling `handle` or doing any other additional actions.

For this, we'll build **events** and an **event broker**. Events will be discussed further in the *Observer Design Pattern* lesson; the implementation here will be a very basic one.

We'll also need to apply **Command Query Separation**: we will have different subsystems to handle **commands** and **queries** separately. We'll focus on queries; Commands will be discussed further in the *Command Design Pattern* lesson.

Let's begin with implementing events:

In [1]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

An `Event` is essentially a list of functions that we can call.

Let's re-implement the `Creature` class with a few changes: we will initialize it with a reference to the event broker, which will take care of the chain of responsibility. We'll call the event broker `game`, because every creature is part of a game.

In [ ]:
class Creature:
    def __init__(self, game, name, attack, defense):
        self.initial_defense = defense
        self.initial_attack = attack
        self.name = name
        self.game = game

Note that we've also changed `defense` to `initial_defense` and `attack` to `initial_attack`, because we will rely on the modifiers to change these initial values. In order to do this, we need to implement a query mechanism for getting the creature's values, incorporating any modifiers that might exist.

Our `Creature` class is not finished yet; we'll come back to it later. Let's implement our event broker:

In [2]:
class Game:
    def __init__(self):
        self.queries = Event()

    def perform_query(self, sender, query):
        self.queries(sender, query)

Essentially, we're creating an `Event` that everyone can subscribe to whenever someone sends a query: the idea being that if someone, for example, sends a query for a creature's attack, the modifiers can listen to this event and they can modify the returning value.

`perform_query`, as it name implies, performs the actual query: it takes the sender (who's actually asking the query) and the query itself, and then you invoke on the `Event` (which, as you may recall, is a list of functions).

Finally, we need to implement the `Query` class itself:

In [4]:
from enum import Enum

class WhatToQuery(Enum):
    ATTACK = 1
    DEFENSE = 2

class Query:
    def __init__(self, creature_name, what_to_query, default_value):
        self.value = default_value  # name change!
        self.what_to_query = what_to_query
        self.creature_name = creature_name

Note that even though we're providing a `default_value` argument, internally we're renaming it to `value`, because creature modifiers are going to modify this value and they might return something completely different. In other words: our query is not storing the default value, but rather the actual value that subsequently other handlers of the event can modify.

Let's go back to the `Creature` class. Instead of simply returning the attack and defense values, we'll implement getters that will perform queries that will calculate the actual vale to return:

In [5]:
class Creature:
    def __init__(self, game, name, attack, defense):
        self.initial_defense = defense
        self.initial_attack = attack
        self.name = name
        self.game = game

    @property
    def attack(self):
        q = Query(self.name, WhatToQuery.ATTACK, self.initial_attack)
        self.game.perform_query(self, q)
        return q.value

    @property
    def defense(self):
        q = Query(self.name, WhatToQuery.DEFENSE, self.initial_attack)
        self.game.perform_query(self, q)
        return q.value

    def __str__(self):
        return f'{self.name} ({self.attack}/{self.defense})' # We're using the getters rather than the initial values!

Let's see what we have so far in action:

In [6]:
game = Game()
goblin = Creature(game, 'Strong Goblin', 2, 2)
print(goblin)

Strong Goblin (2/2)


Our event broker and query subsystem are now online! Let's build the actual modifiers, once again by creating a base class that other modifiers can inherit from:

In [7]:
from abc import ABC

class CreatureModifier(ABC):
    def __init__(self, game, creature):
        self.creature = creature
        self.game = game
        self.game.queries.append(self.handle) # 1

    def handle(self, sender, query): # 2
        pass

There are a few changes compared to our original `CreatureModifier`:
1. We need to handle the queries event ourselves because we want to be able to change the queries themselves, so that the returned value is modified. We do this by appending our `handle` method to the query.
2. Since we're building an abstract base class, we simply pass on `handle` and let our actual modifier implementations handle it.

Let's create an actual modifier:

In [8]:
class DoubleAttackModifier(CreatureModifier):
    def handle(self, sender, query):
        if (sender.name == self.creature.name and
                query.what_to_query == WhatToQuery.ATTACK):
            query.value *= 2

For this modifier, we simply check that the query is referring to our creature, we check that the query is referring to attacking, and if the conditions are met, we modify the query by doubling the attack value.

Also, since `handle` is being called when intiializing the instance, the modifier will be applied instantly as soon as it's built. Let's see it in action:

In [9]:
game = Game()
goblin = Creature(game, 'Strong Goblin', 2, 2)
print(goblin)

dam = DoubleAttackModifier(game, goblin)
print(goblin)

Strong Goblin (2/2)
Strong Goblin (4/2)


Cool! But we can do a small improvement yet by giving a lifetime to the modifiers: using the `with` keyword, we can define scope and the modifiers will only be applied within that scope. We can do this by implementing the `__enter__` and `__exit__` methods in our base class:

In [10]:
class CreatureModifier(ABC):
    def __init__(self, game, creature):
        self.creature = creature
        self.game = game
        self.game.queries.append(self.handle)

    def handle(self, sender, query):
        pass

    def __enter__(self): # What happens when we enter the `with` block
        return self

    def __exit__(self, exc_type, exc_val, exc_tb): # What happens when we exit the `with` block
        self.game.queries.remove(self.handle) # we unsubscribe

# we need to rerun our modifier implementations; we're also adding a new one as a bonus.
class DoubleAttackModifier(CreatureModifier):
    def handle(self, sender, query):
        if (sender.name == self.creature.name and
                query.what_to_query == WhatToQuery.ATTACK):
            query.value *= 2

class IncreaseDefenseModifier(CreatureModifier):
    def handle(self, sender, query):
        if (sender.name == self.creature.name and
                query.what_to_query == WhatToQuery.DEFENSE):
            query.value += 3

Let's check how this would now work:

In [11]:
game = Game()
goblin = Creature(game, 'Strong Goblin', 2, 2)
print(goblin)

with DoubleAttackModifier(game, goblin):
    print(goblin)
    with IncreaseDefenseModifier(game, goblin):
        print(goblin)

print(goblin)

Strong Goblin (2/2)
Strong Goblin (4/2)
Strong Goblin (4/5)
Strong Goblin (2/2)


As you can see, the modifiers only apply within their scope, and we can now easily create a hierarchy of modifiers.